# **morphOMICs_v2 Tutorial Notebook:</br>Classical morphometric and Sholl analysis**
## Author: **Ryan Cubero**
### For questions, please contact <a href="mailto:rcubero@ist.ac.at">Ryan</a> <br/> For bugs and feature request, please go to the <a href="https://github.com/siegert-lab/MorphOMICs/issues">morphOMICs GitHub issues page</a> <br/>
#### In this notebook, we will go through the modular implementation of <code>morphOMICs</code> to look into morphologies using either classical morphometric or Sholl analysis. Note that this tutorial assumes that you have run <code>Input</code> on your .swc files as done in  <span style="color:#424242">**morphOMICs_v2_Tutorial.ipynb**</span>.

In [1]:
%load_ext autoreload
%autoreload 2

# see if you have nb-black installed
# if not: pip install nb-black
%load_ext lab_black

In [2]:
# change directory to where Morphomics is located
# do this only when you have not installed morphOMICs
import os

os.chdir("/media/drive_siegert/RyCu/Projects/Git_codes/morphomics_dev/morphomics_v2.1")

<br> </br>
#### First, let's import all the necessary libraries to make the notebook work.

In [3]:
from morphomics import protocols
import tomli

2023-06-06 10:47:28.607762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 10:47:28.718872: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-06 10:47:28.718884: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-06 10:47:29.190594: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

####  The central feature of <code>morphOMICs_v2</code> is the ***parameters file***. You can check out some examples of the parameter file in **<span style="color:#424242">morphOMICs_v2/Parameter_files</span>**. It is  written in <a href="https://toml.io/en/">TOML</a> which is then parsed into Python as a dictionary.

<br></br>
#### First, let's define the <code>parameters</code> dictionary by calling the <code>Protocols</code>.

#### <code>Protocols</code> defines the sequence of modules that will be executed and can be used to create workflows. You can choose the following executables: <code>Input</code>, <code>Load_data</code>, <code>Clean_frame</code>, <code>Bootstrap</code>, <code>Persistence_Images</code>, <code>UMAP</code>, <code>Palantir</code>, <code>Plotting</code>, <code>Mapping</code>, <code>Sholl_curves</code>, <code>Morphometrics</code>, <code>Clear_morphframe</code>.

#### In the Protocol below, we shall perform the classical morphology analysis. First, we will load the previously-processed data into <code>morphOMICs_v2</code> using <code>Load_data</code>. Then, we will <code>Clean_frame</code> to drop certain conditions that were loaded but are not needed for downstream analysis or rename some conditions. Then, we will calculate <code>Sholl_curves</code> and <code>Morphometrics</code>.

In [4]:
parameters = tomli.loads(
    """
# sequential list of executables
# Choices of executables: Input, Load_data, Clean_frame, Bootstrap, Persistence_Images, UMAP, Palantir, Plotting, Mapping, Clear_morphframe, Sholl_curves, Morphometrics 
# This is an example of a standard morphOMICs pipeline to obtain the morphological spectrum
Protocols = [
        "Load_data",
        "Clean_frame",
        "Bootstrap",
        "Persistence_Images",
        "UMAP",
        "Plotting",
    ]
"""
)

<br> </br>
#### Now, we initialize the <code><span style="color:#ff8a80">_morphOMICs</span></code> class using <code>protocols.Protocols()</code>. Here, we set the <code><span style="color:#ff8a80">Parameters_ID</span></code> to any number. In my use-case, <code><span style="color:#ff8a80">Parameters_ID</span></code>  serves as an indicator for the set of parameters that I am using, which I write down on my notes. It helps make my results reproducible.

In [5]:
Parameters_ID = 3
_morphOMICs = protocols.Protocols(parameters, Parameters_ID)

# It will output the filename prefix for the files that will be subsequently saved

Unless you have specified the file prefix in the succeeding executables, 
this will be the file prefix: Morphomics.PID3


<br></br>
#### Then, we start with loading the data. <code><span style="color:#ff8a80">_morphOMICs.parameters</span></code> contains the parameters that we have initiated before and we should update it with <code>Input</code> parameters. I have placed comments on what information is needed.

In [6]:
# parameters to load 3D morphological reconstructions
_morphOMICs.parameters["Load_data"] = tomli.loads(
    """
# path to where the input files are located
"folderpath_to_data" = "/media/drive_siegert/RyCu/Projects/TMD/_Data/_TMD_barcodes/Retina_sample"
"filename_prefix" = "Morphomics.PID1.TMD-radial_distances"

# how the input files are separated
"separated_by" = "Model"
"conditions_to_include" = ["Ctrl_Kxa4h","Ctrl_Kxa48h",]

# dictionary key where to store the loaded data
"morphoframe_name" = "morphoframe"
"""
)

#### Let's run <code>Load_data</code>!

In [7]:
_morphOMICs.Load_data()

...loading Ctrl_Kxa4h
...loading Ctrl_Kxa48h


#### <code>Input</code> returns the instance <code><span style="color:#ff8a80">_morphOMICs.morphoframe</span></code> which is a dictionary of pandas DataFrame with keys indicated by <code><span style="color:#ff8a80">_morphOMICs.morphoframe[_morphOMICs.parameters["*Input*"]["*morphoframe_name*"]]</span></code> which contains the *TMD barcodes* (**Barcodes**).

In [8]:
_morphOMICs.morphoframe["morphoframe"].head()

,Region,Condition,Model,Time,Sex,Animal,_files,Filenames,Morphologies,Barcodes
0,IPL,Adulthood,Ctrl_Kxa4h,Adult,F,CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL,Filament_005_IPL_cleared_Trace_0000_nl_correct...,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,<morphomics.Neuron.Neuron.Neuron object at 0x7...,"[[20.75804684935549, 17.748095052710422], [14...."
1,IPL,Adulthood,Ctrl_Kxa4h,Adult,F,CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL,Filament_005_IPL_cleared_Trace_0001_nl_correct...,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,<morphomics.Neuron.Neuron.Neuron object at 0x7...,"[[78.46860149130009, 76.54534465922923], [57.4..."
2,IPL,Adulthood,Ctrl_Kxa4h,Adult,F,CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL,Filament_005_IPL_cleared_Trace_0002_nl_correct...,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,<morphomics.Neuron.Neuron.Neuron object at 0x7...,"[[34.33316491382563, 34.87596242399507], [26.1..."
3,IPL,Adulthood,Ctrl_Kxa4h,Adult,F,CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL,Filament_005_IPL_cleared_Trace_0003_nl_correct...,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,<morphomics.Neuron.Neuron.Neuron object at 0x7...,"[[122.92604691032585, 119.19617370956183], [66..."
4,IPL,Adulthood,Ctrl_Kxa4h,Adult,F,CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL,Filament_005_IPL_cleared_Trace_0004_nl_correct...,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,<morphomics.Neuron.Neuron.Neuron object at 0x7...,"[[66.91833092658726, 68.64142038885886], [64.8..."


#### We can also count the number of morphologies in each condition.

In [9]:
_morphOMICs.morphoframe["morphoframe"].pivot_table(
    index=["Region", "Model", "Sex"], aggfunc="size"
)

Region  Model        Sex
IPL     Ctrl_Kxa48h  F      34
                     M      36
        Ctrl_Kxa4h   F      23
                     M      29
OPL     Ctrl_Kxa48h  F      30
                     M      27
        Ctrl_Kxa4h   F      20
                     M      18
dtype: int64

<br></br>
#### Then, we clean out the _morphoframe_ to remove empty morphologies (which corresponds to artifacts during the Imaris semi-automated reconstructions) as well as renaming conditions using <code>Clean_frame</code>.

In [10]:
# criteria for cleaning the input files
_morphOMICs.parameters["Clean_frame"] = tomli.loads(
    """
# if not 0, must contain the filepath to the morphoframe which will then be saved into morphoframe_name
# otherwise, `morphoframe_name` is the morphoframe that will be cleaned up
"morphoframe_filepath" = 0
"morphoframe_name" = "morphoframe"

# remove morphologies if the number of bars is less than the cutoff
"barcode_size_cutoff" = 5

# retain bars whose length satisfy a certain cutoff
# must be an array with three elements, ["greater_than" "less_than" or "within", bar length condition (must be an array if using "within"), "drop" or "keep"]
# the example below keeps bars whose length is greater than 0, and less than 200.
# if not used, comment the elements of `barlength_cutoff` out
"barlength_cutoff" = [ 
#            ["less_than", 0, "drop"],
#            ["greater_than", 200, "drop"], 
#            ["within", [0,200], "keep"],
            ]

# enumerate which conditions will be merged
# must be an array with three elements [a header of the info_frame (is an element of `Input.conditions`),
#                                       a list of conditions that will be merged (must be an array), 
#                                       the new name of the merged conditions]
# if not used, comment the elements of `combine_conditions` out
"combine_conditions" = [
    ["Model", ["Ctrl_Kxa4h", "Ctrl_Kxa48h"], "Ctrl_Kxa"]
]

# enumerate restrictions
# must be an array with three elements [a header of the info_frame (is an element of `Input.conditions`),  
#                                       list of conditions to either drop or keep (must be an array), 
#                                       "drop" or "keep" conditions specified]
# if not used, comment the elements of `restrict_conditions` out
"restrict_conditions" = [
#    ["Region", ["GCL"], "drop"],
#    ["Model", ["rd1","Cx3cr1_het","Ctrl_Iso","Ctrl_Kxa","rd10",], "keep"],
]

# I would advise saving the cleaned data; value is either `true` or `false` (warning: take note that all the letters are in lower case)
"save_data" = true

# location where to save the data
"save_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Data/_TMD_barcodes/Retina_sample"

# if 0, morphOMICs will automatically create a file prefix, i.e., Morphomics.PID[xxx].[barcode_filter].Cleaned.
# Otherwise, this will be used as the file prefix
"file_prefix" = 0
"""
)

In [11]:
_morphOMICs.Clean_frame()

Removing morphologies with barcode size less than 5.00...
Replacing all instances of `['Ctrl_Kxa4h', 'Ctrl_Kxa48h']` in the `Model` morphoframe column with Ctrl_Kxa
Done!


In [12]:
_morphOMICs.morphoframe["morphoframe"].pivot_table(
    index=["Region", "Model", "Sex"], aggfunc="size"
)

Region  Model     Sex
IPL     Ctrl_Kxa  F      56
                  M      65
OPL     Ctrl_Kxa  F      49
                  M      45
dtype: int64

#### Notice that we have combined "Ctrl_Kxa4h" and "Ctrl_Kxa48h" into one condition.

<br></br>
#### Fist, we calculate Sholl curves.

In [13]:
# parameters for bootstrapping
_morphOMICs.parameters["Sholl_curves"] = tomli.loads(
    """
"morphoframe_name" = "morphoframe"
"Empty_indicator" = "Barcodes"

"swc_types" = 0
"Sholl_radius" = 5
"Sholl_colname" = "sholl_curves"

# I would advise saving the data; value is either `true` or `false` (warning: take note that all the letters are in lower case)
"save_data" = true
# path to folder where to store UMAP function and reduced manifold coordinates
"save_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Results/Retina_Morphomics/Retina_barcodes/"
# if 0, morphOMICs will automatically create a file prefix
# Otherwise, this will be used as the file prefix
"file_prefix" = 0
"""
)

In [14]:
_morphOMICs.Sholl_curves()

Calculating Sholl curves with radius 5.00
Sholl curve calculation is now at Filament_005_IPL_cleared_Trace_0000_nl_corrected.swc
Sholl curve calculation is now at Filament_002_IPL_cleared_Trace_0000_nl_corrected.swc
Sholl curve calculation is now at Filament_002_IPL_cleared_Trace_0006_nl_corrected.swc
Sholl curve calculation is now at Filament_003_IPL_cleaned_Trace_0001_nl_corrected.swc
Sholl curve calculation is now at Filament_003_IPL_cleaned_Trace_0011_nl_corrected.swc
Sholl curve calculation is now at Filament_002_IPL_cleaned_Trace_0006_nl_corrected.swc
Sholl curve calculation is now at Filament_003_OPL_cleared_Trace_0000_nl_corrected.swc
Sholl curve calculation is now at Filament_003_OPL_cleared_Trace_0005_nl_corrected.swc
Sholl curve calculation is now at Filament_004_OPL_cleaned_Trace_0002_nl_corrected.swc
Sholl curve calculation is now at Filament_002_IPL_cleared_Trace_0000_nl_corrected.swc
Sholl curve calculation is now at Filament_002_IPL_cleared_Trace_0011_nl_corrected.swc
S

/home/rcubero/anaconda3/lib/python3.9/site-packages/pandas/core/internals/construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


#### <code>Sholll_curves</code> returns the instance <code><span style="color:#ff8a80">_morphOMICs.metadata[_morphOMICs.parameters["*Sholl_curves*"]["*"Sholl_colname"*"]]</span></code>, which is a 2D-array where each element is (_Sholl radius_, _number of microglial process intersection at the Sholl radius_).

In [15]:
_morphOMICs.metadata["sholl_curves"].head()

,Files,Sholl_curves
0,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,"[[0, 6], [5, 8], [10, 12], [15, 9], [20, 12], ..."
1,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,"[[0, 7], [5, 7], [10, 12], [15, 9], [20, 17], ..."
2,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,"[[0, 6], [5, 10], [10, 13], [15, 13], [20, 14]..."
3,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,"[[0, 6], [5, 8], [10, 9], [15, 7], [20, 7], [2..."
4,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,"[[0, 7], [5, 10], [10, 10], [15, 15], [20, 17]..."


<br></br>
#### Then, we calculate classical morphometric quantities.

In [16]:
# parameters for bootstrapping
_morphOMICs.parameters["Morphometrics"] = tomli.loads(
    """
"morphoframe_name" = "morphoframe"
"Empty_indicator" = "Barcodes"
"temp_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Tutorials/_morphOMICs_v2_Tutorials/tmp"
"Morphometric_colname" = "morphometrics"
"Lmeasure_functions" = [
    ['Length', 'TotalSum'],
    ['N_tips', 'TotalSum'],
    ['Width', 'TotalSum'],
    ['Height', 'TotalSum'],
    ['Depth', 'TotalSum'],
    ['Partition_asymmetry', 'Average'],
    ['Bif_ampl_local', 'Average'],
    ['Bif_ampl_local', 'Maximum'],
    ['Bif_ampl_remote', 'Average'],
    ['Bif_ampl_remote', 'Maximum'],
    ['Bif_tilt_local', 'Average'],
    ['Bif_tilt_local', 'Maximum'],
    ['Bif_tilt_remote', 'Average'],
    ['Bif_tilt_remote', 'Maximum'],
    ['Bif_torque_local', 'Average'],
    ['Bif_torque_local', 'Maximum'],
    ['Bif_torque_remote', 'Average'],
    ['Bif_torque_remote', 'Maximum'],
    ['Contraction', 'Average'],
    ['Fractal_Dim', 'Average'],
    ['Fractal_Dim', 'Maximum'],
    ['Branch_pathlength', 'Average'],
    ['Branch_Order', 'Maximum'],
    ['Terminal_degree', 'Average'],
    ['PathDistance', 'Maximum'],
    ['Helix', 'Maximum'],
]

# I would advise saving the data; value is either `true` or `false` (warning: take note that all the letters are in lower case)
"save_data" = true
# path to folder where to store UMAP function and reduced manifold coordinates
"save_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Results/Retina_Morphomics/Retina_barcodes/"
# if 0, morphOMICs will automatically create a file prefix
# Otherwise, this will be used as the file prefix
"file_prefix" = 0
"""
)

In [19]:
_morphOMICs.Morphometrics()

Calculating classical morphometric quantities...
All of the functions in Morphometrics.Lmeasure_functions are available in Lmeasure!
Running pyLmeasure with 215 files...
Summarizing morphometric quantities into an array...
Done!


In [20]:
_morphOMICs.metadata["morphometrics"].head()

,Files,Length_TotalSum,N_tips_TotalSum,Width_TotalSum,Height_TotalSum,Depth_TotalSum,Partition_asymmetry_Average,Bif_ampl_local_Average,Bif_ampl_local_Maximum,Bif_ampl_remote_Average,...,Bif_torque_remote_Average,Bif_torque_remote_Maximum,Contraction_Average,Fractal_Dim_Average,Fractal_Dim_Maximum,Branch_pathlength_Average,Branch_Order_Maximum,Terminal_degree_Average,PathDistance_Maximum,Helix_Maximum
0,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,640.221,49.0,73.5449,8441.18,1150.43,0.598745,36.2215,87.2043,78.4169,...,82.7731,175.238,0.907656,1.03953,1.21747,6.87925,10.0,3.20176,87.3818,0.04
1,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,1163.960,80.0,128.7440,8328.97,1150.59,0.662783,43.2234,106.4230,78.9577,...,92.3766,176.665,0.897946,1.04651,1.31518,7.50394,18.0,4.44723,143.1330,0.16
2,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,1096.270,82.0,117.5080,8623.44,1152.81,0.575653,47.4665,107.3540,83.5712,...,85.7508,173.457,0.915404,1.02977,1.18188,6.88975,18.0,3.80698,118.5910,0.04
3,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,1194.780,78.0,161.1620,8618.58,1152.59,0.733360,45.2412,99.7939,83.4126,...,93.0362,177.623,0.909113,1.03810,1.21256,7.90715,19.0,4.65239,161.5320,0.05
4,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,1347.880,91.0,114.5390,8443.70,1153.18,0.699855,43.6058,107.2240,81.7722,...,94.6484,178.487,0.904708,1.04031,1.36694,7.65347,22.0,4.01184,133.2900,0.11


<br></br>
## Congratulations! You have now completed the a classical morphometric analysis!

### A copy of the parameters file is saved in **<span style="color:#424242">morphOMICs_v2/Tutorial/Morpphomics.Parameters.3.toml</span>**